### Import thư viện

In [ ]:
import sys
import os
import numpy as np

# Thêm đường dẫn để import module nếu cần
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

import src.data_processing as dp
import src.models as md

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Tải và Phân tích sơ bộ dữ liệu (Data Loading & Parsing)
Bước đầu tiên là đọc dữ liệu từ file CSV thô. Vì dữ liệu thực tế thường chứa lỗi định dạng, chúng ta sẽ parse từng dòng thủ công để đảm bảo kiểm soát tốt các trường thông tin quan trọng như `user_followers`, `retweets`, `user_verified`, v.v.

In [24]:
# 1. Load dữ liệu thô
data_path = '../data/raw/vaccination_tweets.csv'
raw_rows, header = dp.load_csv(data_path)
print(f"Header: {header}")
print(f"Total rows raw: {len(raw_rows)}")

# 2. Parse từng dòng và trích xuất các cột quan trọng
# Index cột trong file CSV (dựa trên header):
# 4: user_created | 5: user_followers | 6: user_friends | 7: user_favourites 
# 8: user_verified | 9: date | 13: retweets | 14: favorites

parsed_data = []
for line in raw_rows:
    parts = dp.parse_csv_line(line)
    if len(parts) >= 15: # Đảm bảo đủ cột
        parsed_data.append(parts)

parsed_data = np.array(parsed_data)
print(f"Total rows parsed: {len(parsed_data)}")

# 3. Tách các cột dữ liệu sang Numpy Array
# Lưu ý: Dữ liệu lúc này vẫn là String
user_created_raw = parsed_data[:, 4]
user_followers_raw = parsed_data[:, 5]
user_friends_raw = parsed_data[:, 6]
user_verified_raw = parsed_data[:, 8]
tweet_date_raw = parsed_data[:, 9]
retweets_raw = parsed_data[:, 13]

Header: id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
Total rows raw: 21731
Total rows parsed: 6841


## 2. Làm sạch dữ liệu (Data Cleaning)
Dữ liệu mạng xã hội thường chứa nhiều nhiễu.
- **Chuyển đổi kiểu:** Đưa dữ liệu về dạng số (Numeric).
- **Xử lý ngoại lai (Outliers):** Sử dụng phương pháp **IQR (Interquartile Range)** để loại bỏ những tài khoản có số lượng follower quá đột biến (gây nhiễu cho mô hình).

In [25]:
print("--- START DATA CLEANING ---")

# 1. Chuyển đổi sang Numeric
followers = dp.to_numeric_safe(user_followers_raw)
friends = dp.to_numeric_safe(user_friends_raw)
retweets = dp.to_numeric_safe(retweets_raw)

# 2. Loại bỏ ngoại lai (Outliers) cho 'user_followers' dùng IQR
# Dữ liệu follower thường phân phối lệch, IQR tốt hơn Z-score
print(f"Original Followers Mean: {np.nanmean(followers):.2f}")

followers_clean, mask_clean = dp.remove_outliers_iqr(followers)
print(f"Cleaned Followers Mean (IQR): {np.nanmean(followers_clean):.2f}")
print(f"Removed {len(followers) - len(followers_clean)} outliers.")

# Cập nhật lại các mảng khác theo mask đã lọc để đồng bộ dữ liệu
friends_clean = friends[mask_clean]
retweets_clean = retweets[mask_clean]
user_verified_clean = user_verified_raw[mask_clean]
user_created_clean = user_created_raw[mask_clean]
tweet_date_clean = tweet_date_raw[mask_clean]

--- START DATA CLEANING ---
Original Followers Mean: 31786.35
Cleaned Followers Mean (IQR): 842.78
Removed 924 outliers.


## 3. Kỹ thuật đặc trưng (Feature Engineering)
Tạo ra các biến mới mang nhiều ý nghĩa thông tin hơn từ các biến gốc:
* **Account Age (Tuổi tài khoản):** Tính từ ngày tạo (`user_created`) đến ngày đăng tweet. Tài khoản lâu năm thường có uy tín khác biệt.
* **Engagement Ratio (Tỷ lệ tương tác):** Đo lường mức độ "viral" tương đối bằng cách so sánh số `Retweets` với mạng lưới bạn bè (`Friends`).

In [26]:
print("\n--- START FEATURE ENGINEERING ---")

# 1. Tính tuổi tài khoản (Account Age)
# account_age = ngày tweet - ngày tạo account
account_age_days = dp.create_account_age_feature(user_created_clean, tweet_date_clean)

print(f"Account Age created. Sample: {account_age_days[:5]} (days)")

# 2. Tạo tỷ lệ tương tác (Engagement Ratio)
# Ratio = Retweets / Friends (giả sử friends là số người họ follow -> khả năng tiếp cận)
engagement_rate = dp.create_engagement_ratio(retweets_clean, friends_clean)

print(f"Engagement Ratio created. Sample: {engagement_rate[:5]}")


--- START FEATURE ENGINEERING ---
Account Age created. Sample: [4274 4101  170  233  321] (days)
Engagement Ratio created. Sample: [0.        0.0015015 0.        0.        0.       ]


## 4. Xử lý dữ liệu thiếu (Imputation)
Trong thực tế, dữ liệu thường bị khuyết. Tại đây, chúng ta sẽ:
1.  **Giả lập:** Tạo nhân tạo một số giá trị thiếu (NaN) để thử nghiệm.
2.  **Điền khuyết (Imputation):** Thay vì điền bằng Mean/Median đơn giản, ta sử dụng **Linear Regression** để dự đoán giá trị khuyết dựa trên mối tương quan mạnh giữa `Followers` và `Retweets`.

In [27]:
print("\n--- START IMPUTATION ---")

# Giả lập tình huống: Tạo một số giá trị thiếu (NaN) trong cột retweets
retweets_missing_sim = retweets_clean.copy()
np.random.seed(42)
nan_indices = np.random.choice(len(retweets_missing_sim), size=100, replace=False)
retweets_missing_sim[nan_indices] = np.nan

print(f"Simulated missing values: {np.sum(np.isnan(retweets_missing_sim))}")

# Dùng Linear Regression để điền khuyết
# X = Followers (Log transform để tuyến tính hơn), y = Retweets
X_for_impute = dp.log_transformation(followers_clean)
y_imputed = md.impute_linear_regression(X_for_impute, retweets_missing_sim)

print(f"Missing values after imputation: {np.sum(np.isnan(y_imputed))}")
print(f"Sample imputed values: {y_imputed[nan_indices[:5]]}")


--- START IMPUTATION ---
Simulated missing values: 100
Missing values after imputation: 0
Sample imputed values: [0.71806794 0.4048996  0.57631836 1.29394869 0.98099816]


## 5. Chuẩn hóa và Biến đổi dữ liệu (Scaling & Transformation)
Các mô hình học máy hoạt động tốt hơn khi dữ liệu có cùng thang đo hoặc phân phối chuẩn.
* **Log Transformation:** Áp dụng cho `Followers` để giảm độ lệch (skewness) của phân phối đuôi dài.
* **Min-Max Scaling:** Đưa `Account Age` về đoạn `[0, 1]`.
* **Standardization (Z-score):** Đưa `Friends` về phân phối chuẩn tắc ($\mu=0, \sigma=1$).

In [28]:
print("\n--- START SCALING ---")

# 1. Log Transformation: Dùng cho Followers (do phân phối lệch)
followers_log = dp.log_transformation(followers_clean)
dp.describe_data(followers_log, "Log(Followers)")

# 2. Min-Max Scaling: Dùng cho Account Age (đưa về [0, 1])
age_minmax = dp.min_max_scaling(account_age_days)
dp.describe_data(age_minmax, "MinMax(Account Age)")

# 3. Z-score Standardization: Dùng cho Friends (chuẩn bị cho thuật toán Gradient)
friends_std = dp.standardization(friends_clean)
dp.describe_data(friends_std, "Standardized(Friends)")
print(f"Verify Mean ~ 0: {np.mean(friends_std):.5f}")
print(f"Verify Std ~ 1: {np.std(friends_std):.5f}")


--- START SCALING ---
--- Thống kê mô tả: Log(Followers) ---
Mean:   5.6150
Median: 5.7714
Std:    1.7907
Min:    0.0000
Max:    8.5531
------------------------------
--- Thống kê mô tả: MinMax(Account Age) ---
Mean:   0.4634
Median: 0.5259
Std:    0.2784
Min:    0.0000
Max:    1.0000
------------------------------
--- Thống kê mô tả: Standardized(Friends) ---
Mean:   0.0000
Median: -0.4139
Std:    1.0000
Min:    -0.7651
Max:    3.8616
------------------------------
Verify Mean ~ 0: 0.00000
Verify Std ~ 1: 1.00000


## 6. Kiểm định giả thuyết (Hypothesis Testing)
Trước khi kết thúc, ta thực hiện một kiểm định thống kê để xác nhận các giả định về dữ liệu.
* **Giả thuyết:** Liệu có sự khác biệt thực sự về lượng người theo dõi (`Followers`) giữa nhóm tài khoản đã xác minh (**Verified**) và chưa xác minh (**Unverified**)?
* **Phương pháp:** Sử dụng **T-test** (Welch's T-test).

In [29]:
print("\n--- START HYPOTHESIS TESTING ---")

# Tách nhóm Verified và Unverified
is_verified = (user_verified_clean == 'True')

verified_followers = followers_clean[is_verified]
unverified_followers = followers_clean[~is_verified]

print(f"Số lượng Verified: {len(verified_followers)}")
print(f"Số lượng Unverified: {len(unverified_followers)}")

# Thực hiện T-test (Welch's T-test)
# H0: Mean Followers (Verified) = Mean Followers (Unverified)
# H1: Mean Followers (Verified) != Mean Followers (Unverified)

t_stat = dp.perform_ttest_ind(verified_followers, unverified_followers)


--- START HYPOTHESIS TESTING ---
Số lượng Verified: 111
Số lượng Unverified: 5806
Kiểm định T-test (Welch's):
   H0: Mean Group 1 == Mean Group 2
   H1: Mean Group 1 != Mean Group 2
   Mean 1: 3208.95, Mean 2: 797.54
   T-statistic: 18.0679
   Kết luận (mức ý nghĩa 5%): Bác bỏ H0 (Khác biệt có ý nghĩa)


## 7. Đóng gói và Lưu trữ (Export)
Cuối cùng, ghép tất cả các đặc trưng đã xử lý thành một ma trận dữ liệu duy nhất và lưu dưới dạng `.csv` tại thư mục `processed`. Dữ liệu này sẽ là đầu vào cho notebook Huấn luyện mô hình tiếp theo.

In [30]:
# Đóng gói dữ liệu sạch
processed_data = {
    'followers_log': followers_log,
    'friends_std': friends_std,
    'account_age_norm': age_minmax,
    'retweets_imputed': y_imputed,
    'label_verified': is_verified.astype(int)
}

# Chuyển mỗi biến thành mảng 2D (n, 1) nếu đang là 1D
processed_arrays = []
for name, arr in processed_data.items():
    arr = np.asarray(arr)
    if arr.ndim == 1:
        arr = arr.reshape(-1, 1)
    processed_arrays.append(arr)

# Ghép theo chiều cột
merged = np.hstack(processed_arrays)

# Lưu thành CSV
output_file = os.path.join('../data/processed/', "processed_dataset.csv")
np.savetxt(output_file, merged, delimiter=",")

print(f"\nSaved merged CSV to: {output_file}")


Saved merged CSV to: ../data/processed/processed_dataset.csv
